<a href="https://colab.research.google.com/github/Nami-59/VK/blob/main/VK_friends_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vk_api

In [3]:
import pandas as pd
import vk_api

In [35]:
vkApiSession= vk_api.VkApi(token="token")
vk = vkApiSession.get_api()

In [5]:
def friends(id):
  try:
    m=vk.friends.get(user_id = str(id), count = 50000)
    for q in m['items']:
       with open('friends.txt','a') as f2:
           f2.write(str(id) + ';' + str(q) + '\n')
  except:
    with open('friends.txt','a') as f2:
        f2.write(str(id) + ';0\n')

In [21]:
my_id = pd.read_csv('id_group.txt', header=None) #первый слой

In [22]:
for i in range(0, len(my_id)):
  friends(my_id[0][i]) #второй слой

In [25]:
data = pd.read_csv('friends.txt', sep=';',header=None)

In [28]:
data1 = data[1].unique()
data1 = [x for x in data1 if x not in set(my_id)]

In [29]:
for j in range(0, len(data1)):
  friends(data1[j]) #третий слой

In [24]:
data_1 = pd.read_csv('friends.txt', sep=';',header=None)
for j in range(0, len(data_1)):
  if data_1[1][j] == 0:
    data_1 = data_1.drop(j)

In [25]:
data2 = data_1[1].unique()
data_2 = data_1[0].unique()
data2 = [x for x in data2 if x not in set(data_2)]

In [8]:
def get_indices(lst, el):
    t = list()
    for i in range(0, len(lst)):
        if lst[i] == el:
            t.append(i)
    return t

In [90]:
def are_friends(df, i):
  k = list()
  v = 0
  id = list()
  for j in range(0, 2000, 1000):
  #for j in range(0,780000,1000):
    v+=1000
    m=vk.friends.areFriends(user_ids = [df[i], df[j:v]])
    for w in range(0,len(m)):
      k.append(m[w]['friend_status'])
      id.append(m[w]['user_id'])
  return k, id

In [52]:
def are_friends_78(df, i):
  k = list()
  id = list()
  m=vk.friends.areFriends(user_ids = [df[i], df[2000:len(df)]])
  #m=vk.friends.areFriends(user_ids = [df[i], df[781000:len(df)]])
  for j in range(0,len(m)):
    k.append(m[j]['friend_status'])
    id.append(m[j]['user_id'])
  return k, id

In [45]:
def friends_friends(df):
  for j in range(0, len(df)-1):
    print(j)
    k, id = are_friends(df, j)
    k1, id1 = are_friends_78(df, j)
    k+=k1
    id+=id1
    i = get_indices(k, 3)
    for v in range(0,len(i)):
      with open('friends_dop.txt','a') as f2:
        f2.write(str(df[j]) + ';' + str(id[i[v]]) + '\n')

In [ ]:
friends_friends(data2)

Центральности

In [225]:
import networkx as nx

In [ ]:
data = pd.read_csv('friends.txt', sep=';',header=None)
data_3 = pd.read_csv('friends_dop.txt', sep=';',header=None)
df = data.append(data_3, ignore_index = True)

In [96]:
for j in range(0, len(df)):
  if df[1][j] == 0:
    df = df.drop(j)

In [97]:
G = nx.Graph()
nodes = df[1].unique()
edges = df.values
G.add_nodes_from(nodes)
G.add_edges_from(edges)
#len(nodes), len(edges)

In [98]:
my_id = pd.read_csv('id_group.txt', header=None)

In [179]:
def centr(df, id):
  df1 = list(df.items())
  line = list()
  id_group = list()
  for i in range(0, len(id)):
    for j in range(0, len(df1)):
      if id[0][i] == df1[j][0]:
          id_group.append(my_id[0][i])
          line.append(df1[j][1])
          break
  return id_group, line

In [130]:
close_centr = nx.closeness_centrality(G)

In [182]:
id_group_close, line_close = centr(close_centr, my_id)

In [170]:
between_centr = nx.betweenness_centrality(G)

In [183]:
id_group_between, line_between = centr(between_centr, my_id)

In [176]:
eigen_centr = nx.eigenvector_centrality(G, max_iter = 1000)

In [184]:
id_group_eigen, line_eigen = centr(eigen_centr, my_id)

In [199]:
table = pd.DataFrame({
    # 'id_group_close': id_group_close,
     'closeness_centrality': line_close,
    # 'id_group_between': id_group_between,
     'betweenness_centrality': line_between,
    # 'id_group_eigen': id_group_eigen,
     'eigenvector_centrality': line_eigen
}, index=id_group_close)
table.index.name = 'id'
table

,closeness_centrality,betweenness_centrality,eigenvector_centrality
id,,,
224364474,0.356485,0.000149,0.013098
256804252,0.381672,0.017644,0.140279
212487510,0.393819,0.008854,0.094531
531619927,0.343469,0.000478,0.020047
194848002,0.350741,0.000470,0.020413
381907905,0.364103,0.000064,0.009434
444639273,0.375661,0.001517,0.028390
308412461,0.502613,0.000085,0.020303
232210943,0.342798,0.000793,0.026102


In [224]:
#Центры в группе
max_centr = pd.DataFrame({
     'id': table.idxmax(),
     'Значение': table.max(),
})
max_centr

,id,Значение
closeness_centrality,308412461,0.502613
betweenness_centrality,256804252,0.017644
eigenvector_centrality,256804252,0.140279


In [215]:
cl = table['closeness_centrality'].sort_values(ascending=False)
bet = table['betweenness_centrality'].sort_values(ascending=False)
eig = table['eigenvector_centrality'].sort_values(ascending=False)

In [223]:
index_max = pd.DataFrame({
     'closeness_centrality_max': cl.index,
     'betweenness_centrality_max': bet.index,
     'eigenvector_centrality_max': eig.index
})
index_max

,closeness_centrality_max,betweenness_centrality_max,eigenvector_centrality_max
0,308412461,256804252,256804252
1,461814307,212487510,212487510
2,276581495,315590903,315590903
3,212487510,315629783,315629783
4,256804252,75785096,75785096
5,184267947,54705450,54705450
6,444639273,444639273,143661083
7,54705450,143661083,139939428
8,146075397,139939428,444639273
9,383087847,232210943,146075397
